18120543 - Trần Đại Tài / 18120532 - Nguyễn Hoàng Sang

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time # Dùng để sleep chương trình
import pandas as pd # Dùng để đọc và hiển thị file csv (Pandas sẽ được học chi tiết ở buổi tới)
import datetime as dt # Dùng để xử lý dữ liệu thời gian
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline





# Đọc file CSV

In [2]:
data_df=pd.read_csv('Laptops_Data_clean1.csv')


# Dữ liệu có bao nhiêu dòng, bao nhiêu cột?

In [3]:
data_df.shape

(1302, 13)

Xem 5 dòng đầu của Datarame

In [4]:
data_df.head()


,Unnamed: 0,Company,Memory_Structure_Categories,Gpu_Type,Gpu_Company,OpSys,Cpu_Company,Inches,CPU_Frequency_Numeric,Ram_Numeric,Size_Numeric,Weight_Numeric,Price_dollars
0,0,Asus,Hybrid,GeForce GTX 1080,Nvidia,Windows 10,Intel,17.3,2.9,32 MB,512MB,4.7,4279$
1,1,Lenovo,Hybrid,GeForce GTX 980M,Nvidia,Windows 10,Intel,17.3,2.7,32 MB,512MB,4.6,3564$
2,2,Lenovo,Hybrid,GeForce GTX 980M,Nvidia,Windows 10,Intel,17.3,2.7,16 MB,256MB,4.6,3025$
3,3,Lenovo,Hybrid,GeForce GTX 1070,Nvidia,Windows 10,Intel,17.3,2.7,32 MB,256MB,4.6,2929$
4,4,Lenovo,Hybrid,GeForce GTX 980M,Nvidia,Windows 10,Intel,17.3,2.6,16 MB,128MB,4.6,2519$


# Xử lý dữ liệu các cột

In [5]:
data_df['Price_dollars'] = data_df['Price_dollars'].map(lambda x: str(x).replace('$',''))
data_df['Ram_Numeric'] = data_df['Ram_Numeric'].map(lambda x: str(x).replace('MB',''))
data_df['Size_Numeric'] = data_df['Size_Numeric'].map(lambda x: str(x).replace('MB',''))
data_df

,Unnamed: 0,Company,Memory_Structure_Categories,Gpu_Type,Gpu_Company,OpSys,Cpu_Company,Inches,CPU_Frequency_Numeric,Ram_Numeric,Size_Numeric,Weight_Numeric,Price_dollars
0,0,Asus,Hybrid,GeForce GTX 1080,Nvidia,Windows 10,Intel,17.3,2.90,32,512,4.70,4279
1,1,Lenovo,Hybrid,GeForce GTX 980M,Nvidia,Windows 10,Intel,17.3,2.70,32,512,4.60,3564
2,2,Lenovo,Hybrid,GeForce GTX 980M,Nvidia,Windows 10,Intel,17.3,2.70,16,256,4.60,3025
3,3,Lenovo,Hybrid,GeForce GTX 1070,Nvidia,Windows 10,Intel,17.3,2.70,32,256,4.60,2929
4,4,Lenovo,Hybrid,GeForce GTX 980M,Nvidia,Windows 10,Intel,17.3,2.60,16,128,4.60,2519
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1297,1297,Samsung,SSD,HD Graphics 620,Intel,Windows 10,Intel,13.3,2.50,8,256,0.81,1649
1298,1298,Lenovo,Flash,HD Graphics 400,Intel,Windows 10,Intel,10.1,1.44,4,64,0.69,711
1299,1299,Lenovo,Flash,HD Graphics 400,Intel,Android,Intel,10.1,1.44,4,64,0.69,604
1300,1300,Lenovo,Flash,HD Graphics 400,Intel,Windows 10,Intel,10.1,1.44,4,64,0.69,527


# Kiểm tra kiểu dữ liệu của các cột

In [6]:
data_df.dtypes

Unnamed: 0                       int64
Company                         object
Memory_Structure_Categories     object
Gpu_Type                        object
Gpu_Company                     object
OpSys                           object
Cpu_Company                     object
Inches                         float64
CPU_Frequency_Numeric          float64
Ram_Numeric                     object
Size_Numeric                    object
Weight_Numeric                 float64
Price_dollars                   object
dtype: object

# Chuyển các cột dạng object về dạng số

In [7]:
data_df['Price_dollars']=data_df['Price_dollars'].astype(float) 
data_df['Ram_Numeric']=data_df['Ram_Numeric'].astype(int)   
data_df['Size_Numeric']=data_df['Size_Numeric'].astype(int)   

In [8]:
data_df.dtypes

Unnamed: 0                       int64
Company                         object
Memory_Structure_Categories     object
Gpu_Type                        object
Gpu_Company                     object
OpSys                           object
Cpu_Company                     object
Inches                         float64
CPU_Frequency_Numeric          float64
Ram_Numeric                      int32
Size_Numeric                     int32
Weight_Numeric                 float64
Price_dollars                  float64
dtype: object

# Kiểm tra xem dữ liệu có bị trùng hay không

In [9]:
data_df.index.duplicated().sum()


0

# Kiểm tra xem Cột Price_dollars có giá trị thiếu hay không?

In [10]:
data_df['Price_dollars'].isna().sum()

9

# Điền vào các ô thiếu giá trị trung bình của cột

In [11]:
data_df['Price_dollars'].fillna(value=data_df['Price_dollars'].mean(), inplace=True)


# Với mỗi cột có kiểu dữ liệu dạng số, các giá trị được phân bố như thế nào?

In [12]:
df = data_df.select_dtypes(include='number')
def missing_ratio(s):
     return (s.isna().mean()*100).round(1)
def median(df):
    return df.quantile(0.5)
def lower_quartile(df):
    return df.quantile(0.25)
def upper_quartile(df):
    return df.quantile(0.75)
num_col_info_df=df.agg([missing_ratio,min,lower_quartile,median,upper_quartile,max])
num_col_info_df

,Unnamed: 0,Inches,CPU_Frequency_Numeric,Ram_Numeric,Size_Numeric,Weight_Numeric,Price_dollars
missing_ratio,0.00,0.0,0.0,0.0,0.0,0.00,0.0
min,0.00,10.1,0.9,2.0,8.0,0.69,191.0
lower_quartile,325.25,14.0,2.0,4.0,256.0,1.50,660.5
median,650.50,15.6,2.5,8.0,256.0,2.04,1077.0
upper_quartile,975.75,15.6,2.7,8.0,512.0,2.30,1634.0
max,1301.00,18.4,3.6,64.0,2000.0,4.70,6709.0


Bảng dưới đây cho ta thấy thị phần của các hãng laptop

In [13]:
data_df['Company'].value_counts(normalize=True) * 100

Lenovo       22.811060
Dell         22.811060
HP           21.044547
Asus         12.135177
Acer          7.910906
MSI           4.147465
Toshiba       3.686636
Apple         1.612903
Samsung       0.614439
Mediacom      0.537634
Razer         0.537634
Microsoft     0.460829
Xiaomi        0.307220
Vero          0.307220
Fujitsu       0.230415
LG            0.230415
Chuwi         0.230415
Google        0.230415
Huawei        0.153610
Name: Company, dtype: float64

Loại bỏ cột Gpu_Type, vì cột này không có giá trị trong huấn luyện

In [14]:
data_df = data_df.drop(["Gpu_Type"], axis=1)

# Tiền xử lý tập train

In [15]:

column_names_for_onehot = data_df[["Company","Cpu_Company","Memory_Structure_Categories","Gpu_Company","OpSys"]].columns[0:]
column_names_for_onehot


Index(['Company', 'Cpu_Company', 'Memory_Structure_Categories', 'Gpu_Company',
       'OpSys'],
      dtype='object')

In [16]:
data_df = pd.get_dummies(data_df, columns=column_names_for_onehot)

In [17]:
data_df

,Unnamed: 0,Inches,CPU_Frequency_Numeric,Ram_Numeric,Size_Numeric,Weight_Numeric,Price_dollars,Company_Acer,Company_Apple,Company_Asus,...,Gpu_Company_Nvidia,OpSys_Android,OpSys_Chrome OS,OpSys_Linux,OpSys_Mac OS X,OpSys_No OS,OpSys_Windows 10,OpSys_Windows 10 S,OpSys_Windows 7,OpSys_macOS
0,0,17.3,2.90,32,512,4.70,4279.0,0,0,1,...,1,0,0,0,0,0,1,0,0,0
1,1,17.3,2.70,32,512,4.60,3564.0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
2,2,17.3,2.70,16,256,4.60,3025.0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
3,3,17.3,2.70,32,256,4.60,2929.0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
4,4,17.3,2.60,16,128,4.60,2519.0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1297,1297,13.3,2.50,8,256,0.81,1649.0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1298,1298,10.1,1.44,4,64,0.69,711.0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1299,1299,10.1,1.44,4,64,0.69,604.0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1300,1300,10.1,1.44,4,64,0.69,527.0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


# Tách các tập

In [18]:
y = data_df["Price_dollars"]
X = data_df.drop(["Price_dollars"], axis=1)
y

0       4279.0
1       3564.0
2       3025.0
3       2929.0
4       2519.0
         ...  
1297    1649.0
1298     711.0
1299     604.0
1300     527.0
1301     351.0
Name: Price_dollars, Length: 1302, dtype: float64

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,test_size=0.3, random_state=1)

# Mô hình linear Regression

In [20]:
model = LinearRegression()

# Fitting our model with all of our features in X
model.fit(X_train, y_train)
#
score = model.score(X_test, y_test)
print(f"R2 Score: {score}")

R2 Score: 0.7408956379734112
